County Voting Data From https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi%3A10.7910%2FDVN%2FVOQCHQ

In [1]:
# Imports
import numpy as np
import pandas as pd

In [2]:
# Read CSV Files
df = pd.read_csv('../data/College Records - college-counties.csv')
cvd = pd.read_csv('../data/countypres_2000-2020.csv')

In [3]:
# Clean input data to remove nulls and reset index
df = df.dropna()
df.reset_index(drop=True, inplace=True)

In [4]:
# Merge liberal and conservative votes in County Votng Data
cvd = cvd[cvd['party']!="OTHER"]
cvd = cvd.reset_index(drop=True)
PARTY_REPLACEMENTS = {"LIBERAL": ["DEMOCRAT", "GREEN"], "CONSERVATIVE": ["REPUBLICAN", "LIBERTARIAN"]}
for party, replacements in PARTY_REPLACEMENTS.items():
    cvd.loc[cvd['party'].isin(replacements), 'party'] = party
grouped = cvd.groupby(['year','state','county_name', 'party'], as_index=False)
cvd = grouped.agg({'candidatevotes':'sum'})
del grouped, PARTY_REPLACEMENTS, party, replacements

In [14]:
COLLEGE = "Harvard"
college_row = df.loc[df['College'] == COLLEGE]
college_row = college_row.reset_index(drop=True)
# Average scores for each election cycle
START_YEAR = 2012
END_YEAR = 2020
# Average the scores for the next 4 years
averages = {}
total = 0
for i in range(START_YEAR, END_YEAR+1):
    if i % 4 == 0 and i != START_YEAR:
        averages[i-4] = total/4
    total += college_row[str(i)][0]
county = college_row['County'][0].upper()
county_data = cvd.loc[(cvd['county_name']==county)]
county_data.reset_index(drop=True, inplace=True)
averages

{2012: -2.83853125, 2016: -6.76952775}